In [47]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

def naiveBayes(df, output):
    model = GaussianNB()
    return cross_val_score(model, df, output, cv=5)

In [48]:
df = pd.read_csv('dataset.csv')
output = df['koi_disposition']
output = output.replace(['FALSE POSITIVE', 'CONFIRMED'], [0,1])
df = df.drop(columns=['kepoi_name', 'koi_disposition'])

In [49]:
naiveBayes(df, output)

0.8033643024391168